In [1]:
# experiment 2

In [ ]:
!pip install kaggle wandb onnx -Uq
from google.colab import drive
drive.mount('/content/drive')

In [3]:
! mkdir ~/.kaggle

In [4]:
!cp /content/drive/MyDrive/ColabNotebooks/kaggle_API_credentials/kaggle.json ~/.kaggle/kaggle.json

In [5]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c challenges-in-representation-learning-facial-expression-recognition-challenge

In [ ]:
! unzip challenges-in-representation-learning-facial-expression-recognition-challenge.zip

In [8]:
!pip install wandb onnx -Uq

# imports

In [9]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from tqdm.auto import tqdm

import wandb


In [ ]:
torch.backends.cudnn.deterministic = True
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)

# Device configuration
device = "cuda" if torch.cuda.is_available() else "cpu" # detect the GPU if any, if not use CPU, change cuda to mps if you have a mac
print("Device available: ", device)

In [ ]:
wandb.login()

In [ ]:
run = wandb.init(
    entity="konstantine25b-free-university-of-tbilisi-",
    project="Facial_Expression_Recognition_2",
    config={
        "learning_rate": 0.001,
        "architecture": "Simpler CNN (2 conv layers + 2 FC layers)",
        "dataset": "Facial Expression Recognition Challenge",
        "epochs": 20,
        "batch_size": 32,
        "optimizer": "Adam",
        "loss_function": "Cross-Entropy Loss",
        "weight_decay": 1e-5,
        "dropout_rate": 0.3,
    },
)

# Data

In [13]:
# Load the original training data
original_train_df = pd.read_csv('train.csv')

# First split: Create a test set (10% of original data)
train_val_df, test_df = train_test_split(original_train_df, test_size=0.1,
                                         random_state=42, stratify=original_train_df['emotion'])

# Second split: Split remaining data into training (80%) and validation (20%)
train_df, val_df = train_test_split(train_val_df, test_size=0.2,
                                    random_state=42, stratify=train_val_df['emotion'])

# Define emotion labels
emotion_labels = {0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy',
                  4: 'Sad', 5: 'Surprise', 6: 'Neutral'}


In [ ]:
print(f"Original data size: {len(original_train_df)}")
print(f"Training set size: {len(train_df)} ({len(train_df)/len(original_train_df)*100:.1f}%)")
print(f"Validation set size: {len(val_df)} ({len(val_df)/len(original_train_df)*100:.1f}%)")
print(f"Test set size: {len(test_df)} ({len(test_df)/len(original_train_df)*100:.1f}%)")

# Print class distribution in each set
print("\nEmotion distribution:")
for i, emotion in emotion_labels.items():
    train_count = sum(train_df['emotion'] == i)
    val_count = sum(val_df['emotion'] == i)
    test_count = sum(test_df['emotion'] == i)
    print(f"  {emotion}: Train={train_count}, Val={val_count}, Test={test_count}")


# model building

In [15]:
class FER2013Dataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        pixels = [int(pixel) for pixel in row['pixels'].split()]
        image = np.array(pixels, dtype=np.uint8).reshape(48, 48)

        # Convert to PIL Image for transforms
        image = Image.fromarray(image)

        if self.transform:
            image = self.transform(image)

        label = row['emotion']
        return image, label

In [16]:
train_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Validation and test sets only need basic transformations
val_test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Create datasets with appropriate transforms
train_dataset = FER2013Dataset(train_df, transform=train_transform)
val_dataset = FER2013Dataset(val_df, transform=val_test_transform)
test_dataset = FER2013Dataset(test_df, transform=val_test_transform)

overfit test

In [17]:
indices = list(range(20))
overfit_dataset = torch.utils.data.Subset(train_dataset, indices)
batch_size = wandb.config.batch_size
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
overfit_loader = DataLoader(overfit_dataset, batch_size=batch_size, shuffle=True)

# Visualize some sample images
def pixels_to_image(pixels_str):
    pixels = [int(pixel) for pixel in pixels_str.split()]
    image = np.array(pixels).reshape(48, 48)
    return image

In [ ]:
plt.figure(figsize=(14, 3))
for i, emotion in emotion_labels.items():
    sample = train_df[train_df['emotion'] == i].iloc[0]
    img = pixels_to_image(sample['pixels'])

    plt.subplot(1, 7, i+1)
    plt.imshow(img, cmap='gray')
    plt.title(emotion)
    plt.axis('off')

plt.tight_layout()
plt.savefig('emotion_samples.png')
plt.show()

# Log the examples to wandb
wandb.log({"emotion_samples": wandb.Image('emotion_samples.png')})


In [19]:
class SimplerCNN(nn.Module):
    def __init__(self, dropout_rate=0.3):
        super(SimplerCNN, self).__init__()


        self.conv1 = nn.Conv2d(1, 32, kernel_size=5, padding=2)
        self.bn1 = nn.BatchNorm2d(32)
        self.pool1 = nn.MaxPool2d(kernel_size=2)


        self.conv2 = nn.Conv2d(32, 64, kernel_size=5, padding=2)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool2 = nn.MaxPool2d(kernel_size=2)


        self.fc1 = nn.Linear(64 * 12 * 12, 128)
        self.dropout = nn.Dropout(dropout_rate)
        self.fc2 = nn.Linear(128, 7)

    def forward(self, x):

        x = self.pool1(F.relu(self.bn1(self.conv1(x))))

        x = self.pool2(F.relu(self.bn2(self.conv2(x))))

        x = x.view(-1, 64 * 12 * 12)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)

        return x

In [ ]:
model = SimplerCNN(dropout_rate=wandb.config.dropout_rate).to(device)
print(model)

In [21]:
wandb.watch(model, log="all")

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(
    model.parameters(),
    lr=wandb.config.learning_rate,
    weight_decay=wandb.config.weight_decay
)

# Learning rate scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode='min',
    factor=0.5,
    patience=3,
    verbose=True
)

In [23]:
def compute_accuracy(loader, model, device):
    model.eval()
    correct = 0
    total = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = 100 * correct / total
    return accuracy, all_preds, all_labels

In [ ]:
print("Testing model on 20 samples to check for overfitting capability...")
epochs_overfit = 30
overfit_losses = []
overfit_accs = []

In [ ]:
for epoch in range(epochs_overfit):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in overfit_loader:
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Calculate accuracy
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    overfit_loss = running_loss / len(overfit_loader)
    overfit_acc = 100 * correct / total
    overfit_losses.append(overfit_loss)
    overfit_accs.append(overfit_acc)

    print(f"Overfit Epoch {epoch+1}/{epochs_overfit}, Loss: {overfit_loss:.4f}, Acc: {overfit_acc:.2f}%")


In [ ]:
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(overfit_losses)
plt.title('Overfitting Test - Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')

plt.subplot(1, 2, 2)
plt.plot(overfit_accs)
plt.title('Overfitting Test - Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')

plt.tight_layout()
plt.savefig('overfit_test.png')
plt.show()

wandb.log({"overfit_test": wandb.Image('overfit_test.png')})

In [ ]:
if max(overfit_accs) > 95:
    print("Model passed the overfitting test! Proceeding with full training.")
else:
    print("Warning: Model may have issues with gradient flow as it didn't achieve high accuracy on the small dataset.")


In [28]:
model = SimplerCNN(dropout_rate=wandb.config.dropout_rate).to(device)
optimizer = optim.Adam(
    model.parameters(),
    lr=wandb.config.learning_rate,
    weight_decay=wandb.config.weight_decay
)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode='min',
    factor=0.5,
    patience=3,
    verbose=True
)

# Full training loop
num_epochs = wandb.config.epochs
best_val_acc = 0
best_model_path = 'best_model.pth'

In [ ]:
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    # Training phase
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")
    for images, labels in progress_bar:
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Calculate accuracy
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # Update progress bar
        progress_bar.set_postfix({
            'loss': loss.item(),
            'acc': 100 * correct / total
        })

    train_loss = running_loss / len(train_loader)
    train_acc = 100 * correct / total

    # Validation phase
    val_acc, _, _ = compute_accuracy(val_loader, model, device)
    val_loss = 0.0

    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            val_loss += criterion(outputs, labels).item()

    val_loss /= len(val_loader)

    # Update learning rate scheduler
    scheduler.step(val_loss)

    # Log metrics to wandb
    wandb.log({
        "epoch": epoch + 1,
        "train_loss": train_loss,
        "train_accuracy": train_acc,
        "val_loss": val_loss,
        "val_accuracy": val_acc,
        "learning_rate": optimizer.param_groups[0]['lr']
    })

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%, "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")

    # Save the best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), best_model_path)
        print(f"New best model saved with validation accuracy: {val_acc:.2f}%")


In [ ]:
model.load_state_dict(torch.load(best_model_path))
wandb.save(best_model_path)

In [ ]:
test_acc, test_preds, test_labels = compute_accuracy(test_loader, model, device)
print(f"Test Accuracy: {test_acc:.2f}%")
wandb.log({"test_accuracy": test_acc})

In [ ]:
cm = confusion_matrix(test_labels, test_preds)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=list(emotion_labels.values()),
            yticklabels=list(emotion_labels.values()))
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.savefig('confusion_matrix.png')
plt.show()

# Log confusion matrix to wandb
wandb.log({"confusion_matrix": wandb.Image('confusion_matrix.png')})


In [ ]:

# Classification report
report = classification_report(test_labels, test_preds,
                               target_names=list(emotion_labels.values()))
print(report)

# Visualize sample predictions
plt.figure(figsize=(15, 10))
test_samples = 5
for i in range(test_samples):
    # Get random test image
    idx = np.random.randint(0, len(test_dataset))
    img, label = test_dataset[idx]

    # Make prediction
    img_tensor = img.unsqueeze(0).to(device)
    with torch.no_grad():
        output = model(img_tensor)
        _, predicted = torch.max(output, 1)

    # Display image with true and predicted labels
    plt.subplot(1, test_samples, i+1)
    plt.imshow(img.squeeze().cpu().numpy(), cmap='gray')
    plt.title(f"True: {emotion_labels[label]}\nPred: {emotion_labels[predicted.item()]}")
    plt.axis('off')

plt.tight_layout()
plt.savefig('sample_predictions.png')
plt.show()

wandb.log({"sample_predictions": wandb.Image('sample_predictions.png')})

# Finish the wandb run
wandb.finish()